#IMPORT

In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
economic_df=pd.read_csv("drive/My Drive/Master Thesis/Imputation/Filled/lCubicImputation_df.csv")
economic_df= economic_df.apply(lambda x: np.log(x) if np.issubdtype(x.dtype, np.number) else x)     #if you want to get LN of data inplace
economic_df.drop(columns=['Date','b1','b3'], inplace=True)
f=[col for col in economic_df.columns]
f

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['b2',
 'b4',
 'b5',
 'b6',
 'b7',
 'b8',
 'b9',
 'b10',
 'b14',
 'b17',
 'b18',
 'B-23',
 'B-24']

#1 Feature

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

tf.keras.utils.set_random_seed(2023)
df=economic_df
# Modify the list of columns accordingly to include all your features
features_columns = [col for col in df.columns]

# Separate features and target variable (if you have one) in your dataset
X = df[features_columns].values

y = df['b2'].values # Replace 'target' with the name of your target column

# Split the data into train, test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=(2/3),shuffle=False)
print(len(X_train),len(X_val),len(X_test))
# Reshape input data to fit the LSTM model
num_features = len(features_columns)

# Grid search hyperparameters
best_r_squared = -np.inf
best_lstm_units = 0
best_epochs = 0
best_batch_size = 0
best_time_steps = 0

# lstm_units_list = [32, 64, 128]
# epochs_list = [50, 100, 150]
# batch_size_list = [16, 32, 64]
# time_steps_list = [3, 5, 7]
lstm_units_list = [64]
epochs_list = [50]
batch_size_list = [32]
time_steps_list = [10]
# X_train.shape[0]
for lstm_units in lstm_units_list:
    for epochs in epochs_list:
        for batch_size in batch_size_list:
            for time_steps in time_steps_list:
                print(f"Training LSTM units: {lstm_units}, Epochs: {epochs}, Batch size: {batch_size}, Time steps: {time_steps}")

                # Reshape input data to fit the LSTM model
                X_train_lstm = TimeseriesGenerator(X_train, y_train, length=time_steps, sampling_rate=1)
                X_val_lstm = TimeseriesGenerator(X_val, y_val, length=time_steps, sampling_rate=1)

                # Create the LSTM model
                model = Sequential()
                model.add(LSTM(lstm_units, activation='sigmoid', input_shape=(time_steps, num_features)))
                model.add(Dense(1)) # Output layer with one node for regression, adjust for classification

                # Compile the model
                model.compile(optimizer='Nadam', loss='mse') # Use appropriate loss for your task

                # Train the model without shuffling
                model.fit(X_train_lstm, epochs=epochs,batch_size=batch_size, shuffle=False)

                # Evaluate the model on the validation set
                y_val_pred = model.predict(X_val_lstm)
                r_squared = r2_score(y_val[time_steps:], y_val_pred)
                print(f"Validation R-squared: {r_squared}")

                # Update best hyperparameters if necessary
                if r_squared > best_r_squared:
                    best_r_squared = r_squared
                    best_lstm_units = lstm_units
                    best_epochs = epochs
                    best_batch_size = batch_size
                    best_time_steps = time_steps

# Train the best model on the combined training and validation set
X_train_combined = np.concatenate((X_train, X_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)
X_train_combined_lstm = TimeseriesGenerator(X_train_combined, y_train_combined , length=best_time_steps, sampling_rate=1)

model = Sequential()
model.add(LSTM(best_lstm_units, activation='sigmoid', input_shape=(best_time_steps, num_features)))
model.add(Dense(1)) # Output layer with one node for regression, adjust for classification
model.compile(optimizer='Nadam', loss='mse')
model.fit(X_train_combined_lstm, epochs=best_epochs, batch_size=best_batch_size, shuffle=False)

# Evaluate the best model on the test set
X_test_lstm = TimeseriesGenerator(X_test, y_test , length=best_time_steps, sampling_rate=1)
y_test_pred = model.predict(X_test_lstm)
test_r_squared = r2_score(y_test[best_time_steps:], y_test_pred)
test_mse = mean_squared_error(y_test[best_time_steps:], y_test_pred)
print(f"\nBest hyperparameters: LSTM units: {best_lstm_units}, Epochs: {best_epochs}, Batch size: {best_batch_size}, Time steps: {best_time_steps}")
print(f"Test R-squared with best hyperparameters: {test_r_squared}")
print(f"Test mse with best hyperparameters: {test_mse}")

117 34 17
Training LSTM units: 64, Epochs: 50, Batch size: 32, Time steps: 10
Epoch 1/50
1/1 [==============================] - 8s 8s/step - loss: 4.1632
Epoch 2/50
1/1 [==============================] - 0s 82ms/step - loss: 3.6036
Epoch 3/50
1/1 [==============================] - 0s 65ms/step - loss: 3.2837
Epoch 4/50
1/1 [==============================] - 0s 64ms/step - loss: 2.9815
Epoch 5/50
1/1 [==============================] - 0s 34ms/step - loss: 2.6979
Epoch 6/50
1/1 [==============================] - 0s 36ms/step - loss: 2.4357
Epoch 7/50
1/1 [==============================] - 0s 34ms/step - loss: 2.1971
Epoch 8/50
1/1 [==============================] - 0s 36ms/step - loss: 1.9839
Epoch 9/50
1/1 [==============================] - 0s 35ms/step - loss: 1.7973
Epoch 10/50
1/1 [==============================] - 0s 35ms/step - loss: 1.6375
Epoch 11/50
1/1 [==============================] - 0s 49ms/step - loss: 1.5041
Epoch 12/50
1/1 [==============================] - 0s 34ms/step 

In [3]:
# Best hyperparameters: LSTM units: 64, Epochs: 50, Batch size: 32, Time steps: 10
# Test R-squared with best hyperparameters: 0.8254605376536703
# Test mse with best hyperparameters: 0.002079440527211053

# #nadam,
# Best hyperparameters: LSTM units: 64, Epochs: 50, Batch size: 32, Time steps: 10
# Test R-squared with best hyperparameters: 0.747716839238484
# Test mse with best hyperparameters: 0.003005668871486755

print(y_test[best_time_steps:])
print(y_test_pred)

[4.04305127 4.12874599 4.1941899  4.24706565 4.27944005 4.33336146
 4.38327585]
[[4.1497254]
 [4.198667 ]
 [4.2398787]
 [4.2827725]
 [4.306782 ]
 [4.326506 ]
 [4.35849  ]]


#All Feature
the result are different from when you run the exact model on one variable, may be its because of the dence layer

In [5]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

tf.keras.utils.set_random_seed(2023)
df=economic_df
# Modify the list of columns accordingly to include all your features
features_columns = [col for col in df.columns]

# Separate features and target variable (if you have one) in your dataset
X = df[features_columns].values

y = df[features_columns].values # Replace 'target' with the name of your target column

# Split the data into train, test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=(2/3),shuffle=False)
print(len(X_train),len(X_val),len(X_test))
# Reshape input data to fit the LSTM model
num_features = len(features_columns)

# Grid search hyperparameters
best_r_squared = -np.inf
best_lstm_units = 0
best_epochs = 0
best_batch_size = 0
best_time_steps = 0

# lstm_units_list = [32, 64, 128]
# epochs_list = [50, 100, 150]
# batch_size_list = [16, 32, 64]
# time_steps_list = [3, 5, 7]
lstm_units_list = [64]
epochs_list = [50]
batch_size_list = [32]
time_steps_list = [10]
# X_train.shape[0]
for lstm_units in lstm_units_list:
    for epochs in epochs_list:
        for batch_size in batch_size_list:
            for time_steps in time_steps_list:
                print(f"Training LSTM units: {lstm_units}, Epochs: {epochs}, Batch size: {batch_size}, Time steps: {time_steps}")

                # Reshape input data to fit the LSTM model
                X_train_lstm = TimeseriesGenerator(X_train, y_train, length=time_steps, sampling_rate=1)
                X_val_lstm = TimeseriesGenerator(X_val, y_val, length=time_steps, sampling_rate=1)

                # Create the LSTM model
                model = Sequential()
                model.add(LSTM(lstm_units, activation='sigmoid', input_shape=(time_steps, num_features)))
                model.add(Dense(num_features)) # Output layer with one node for regression, adjust for classification

                # Compile the model
                model.compile(optimizer='Nadam', loss='mse') # Use appropriate loss for your task

                # Train the model without shuffling
                model.fit(X_train_lstm, epochs=epochs,batch_size=batch_size, shuffle=False)

                # Evaluate the model on the validation set
                y_val_pred = model.predict(X_val_lstm)
                r_squared = r2_score(y_val[time_steps:], y_val_pred)
                print(f"Validation R-squared: {r_squared}")

                # Update best hyperparameters if necessary
                if r_squared > best_r_squared:
                    best_r_squared = r_squared
                    best_lstm_units = lstm_units
                    best_epochs = epochs
                    best_batch_size = batch_size
                    best_time_steps = time_steps

# Train the best model on the combined training and validation set
X_train_combined = np.concatenate((X_train, X_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)
X_train_combined_lstm = TimeseriesGenerator(X_train_combined, y_train_combined , length=best_time_steps, sampling_rate=1)

model = Sequential()
model.add(LSTM(best_lstm_units, activation='sigmoid', input_shape=(best_time_steps, num_features)))
model.add(Dense(num_features)) # Output layer with one node for regression, adjust for classification
model.compile(optimizer='Nadam', loss='mse')
model.fit(X_train_combined_lstm, epochs=best_epochs, batch_size=best_batch_size, shuffle=False)

# Evaluate the best model on the test set
X_test_lstm = TimeseriesGenerator(X_test, y_test , length=best_time_steps, sampling_rate=1)
y_test_pred = model.predict(X_test_lstm)
test_r_squared = r2_score(y_test[best_time_steps:], y_test_pred)
test_mse = mean_squared_error(y_test[best_time_steps:], y_test_pred)
print(f"\nBest hyperparameters: LSTM units: {best_lstm_units}, Epochs: {best_epochs}, Batch size: {best_batch_size}, Time steps: {best_time_steps}")
print(f"Test R-squared with best hyperparameters: {test_r_squared}")
print(f"Test mse with best hyperparameters: {test_mse}")

117 34 17
Training LSTM units: 64, Epochs: 50, Batch size: 32, Time steps: 10
Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 40.1031
Epoch 2/50
1/1 [==============================] - 0s 152ms/step - loss: 39.4725
Epoch 3/50
1/1 [==============================] - 0s 116ms/step - loss: 39.0266
Epoch 4/50
1/1 [==============================] - 0s 169ms/step - loss: 38.5674
Epoch 5/50
1/1 [==============================] - 0s 85ms/step - loss: 38.0926
Epoch 6/50
1/1 [==============================] - 0s 92ms/step - loss: 37.6022
Epoch 7/50
1/1 [==============================] - 0s 114ms/step - loss: 37.0959
Epoch 8/50
1/1 [==============================] - 0s 163ms/step - loss: 36.5736
Epoch 9/50
1/1 [==============================] - 0s 110ms/step - loss: 36.0348
Epoch 10/50
1/1 [==============================] - 0s 105ms/step - loss: 35.4792
Epoch 11/50
1/1 [==============================] - 0s 162ms/step - loss: 34.9065
Epoch 12/50
1/1 [============================

1/1 [==============================] - 0s 203ms/step
Validation R-squared: -978.13185058959
Epoch 1/50
2/2 [==============================] - 2s 13ms/step - loss: 56.4199
Epoch 2/50
2/2 [==============================] - 0s 14ms/step - loss: 55.2289
Epoch 3/50
2/2 [==============================] - 0s 13ms/step - loss: 54.2290
Epoch 4/50
2/2 [==============================] - 0s 14ms/step - loss: 53.1608
Epoch 5/50
2/2 [==============================] - 0s 14ms/step - loss: 52.0236
Epoch 6/50
2/2 [==============================] - 0s 14ms/step - loss: 50.8165
Epoch 7/50
2/2 [==============================] - 0s 14ms/step - loss: 49.5340
Epoch 8/50
2/2 [==============================] - 0s 13ms/step - loss: 48.1655
Epoch 9/50
2/2 [==============================] - 0s 13ms/step - loss: 46.7046
Epoch 10/50
2/2 [==============================] - 0s 13ms/step - loss: 45.1629
Epoch 11/50
2/2 [==============================] - 0s 14ms/step - loss: 43.5672
Epoch 12/50
2/2 [====================

1/1 [==============================] - 0s 198ms/step

Best hyperparameters: LSTM units: 64, Epochs: 50, Batch size: 32, Time steps: 10
Test R-squared with best hyperparameters: -2289.668134128751
Test mse with best hyperparameters: 22.793804718239205


In [7]:
#to check the result
import statistics
y=y_test[best_time_steps:]
yh=y_test_pred

z=[]
for i in range(len(y[0])):
  e=[]
  for j in range(len(y)):
    e.append(y[j][i])
  z.append(e)
print(z)
t=[]
for i in range(len(yh[0])):
  e=[]
  for j in range(len(yh)):
    e.append(yh[j][i])
  t.append(e)
print(t)
r=[]
for i in range(len(t)):
  # print(z[i],t[i])
  r.append(r2_score(z[i],t[i]))
print(r)
print(statistics.mean(r))

[[4.04305126783455, 4.128745988939433, 4.194189897191817, 4.247065649239764, 4.279440045898781, 4.333361462692601, 4.383275854074314], [15.343073530211534, 15.369336495499228, 15.437660232513014, 15.521656363854808, 15.671105926549018, 15.710708777441349, 15.770778457805696], [10.833329096043434, 10.7357172490612, 10.7580970107839, 10.736587635498076, 10.692653139321576, 10.938691348011186, 10.80022638567907], [4.283586561860629, 4.436751534363128, 4.511957804265912, 4.526126978647638, 4.540098189244376, 4.565389315976247, 4.5736795188967205], [8.624611588183507, 8.759511722116487, 8.825412915085566, 8.874168090363968, 8.904630097005011, 8.988446040062405, 9.019058793810718], [8.894670262984233, 8.937875265329263, 9.059168584174444, 9.088850474388472, 9.135401285311689, 9.29596745794362, 9.17378011666438], [9.414097209490203, 9.414097209490203, 10.056453244828615, 10.120496324513569, 10.121429599450872, 10.14680621078697, 10.17640744825082], [10.4691904904112, 10.476019565738097, 10.37

-168.28999931962224

In [ ]:
[4.04305127 4.12874599 4.1941899  4.24706565 4.27944005 4.33336146 4.38327585]
[2.8121562, 2.812847, 2.8143547, 2.8139048, 2.814279, 2.814267, 2.815041]
[[4.1497254],[4.198667 ],[4.2398787],[4.2827725],[4.306782 ],[4.326506 ],[4.35849 ]]

0.7857142857142857